# 50.040 Natural Language Processing (Summer 2020) Homework 2

**Due**


### STUDNET ID:

### Name:

### Students with whom you have discussed (if any):

In [1]:
import copy
from collections import Counter
from nltk.tree import Tree
from nltk import Nonterminal
from nltk.corpus import LazyCorpusLoader, BracketParseCorpusReader
from collections import defaultdict

In [2]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\haoran\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
def set_leave_lower(tree_string):
    if isinstance(tree_string, Tree):
        tree = tree_string
    else:
        tree = Tree.fromstring(tree_string)
    for idx, _ in enumerate(tree.leaves()):
        tree_location = tree.leaf_treeposition(idx)
        non_terminal = tree[tree_location[:-1]]
        non_terminal[0] = non_terminal[0].lower()
    return tree

def get_train_test_data():
    '''
    Load training and test set from nltk corpora
    '''
    train_num = 3900
    test_index = range(10)
    treebank = LazyCorpusLoader('treebank/combined', BracketParseCorpusReader, r'wsj_.*\.mrg')
    cnf_train = treebank.parsed_sents()[:train_num]
    cnf_test = [treebank.parsed_sents()[i+train_num] for i in test_index]
    #Convert to Chomsky norm form, remove auxiliary labels
    cnf_train = [convert2cnf(t) for t in cnf_train]
    cnf_test = [convert2cnf(t) for t in cnf_test]
    return cnf_train, cnf_test
def convert2cnf(original_tree):
    '''
    Chomsky norm form
    '''
    tree = copy.deepcopy(original_tree)
    
    #Remove cases like NP->DT, VP->NP
    tree.collapse_unary(collapsePOS=True, collapseRoot=True)
    #Convert to Chomsky
    tree.chomsky_normal_form()
    
    tree = set_leave_lower(tree)
    return tree

In [4]:
### GET TRAIN/TEST DATA
cnf_train, cnf_test = get_train_test_data()

In [5]:
cnf_train[0].pprint()

(S
  (NP-SBJ
    (NP (NNP pierre) (NNP vinken))
    (NP-SBJ|<,-ADJP-,>
      (, ,)
      (NP-SBJ|<ADJP-,>
        (ADJP (NP (CD 61) (NNS years)) (JJ old))
        (, ,))))
  (S|<VP-.>
    (VP
      (MD will)
      (VP
        (VB join)
        (VP|<NP-PP-CLR-NP-TMP>
          (NP (DT the) (NN board))
          (VP|<PP-CLR-NP-TMP>
            (PP-CLR
              (IN as)
              (NP
                (DT a)
                (NP|<JJ-NN> (JJ nonexecutive) (NN director))))
            (NP-TMP (NNP nov.) (CD 29))))))
    (. .)))


## Question 1

To  better  understand  PCFG,  let’s  consider  the  first  parse  tree  in  thetraining data “cnftrain” as an example.  Run the code we have provided for you and then writedown the roles of.productions(), .rhs(), .lhs(), .leaves()in the ipynb notebook.

In [6]:
rules = cnf_train[0].productions()
print(rules, type(rules[0]))

[S -> NP-SBJ S|<VP-.>, NP-SBJ -> NP NP-SBJ|<,-ADJP-,>, NP -> NNP NNP, NNP -> 'pierre', NNP -> 'vinken', NP-SBJ|<,-ADJP-,> -> , NP-SBJ|<ADJP-,>, , -> ',', NP-SBJ|<ADJP-,> -> ADJP ,, ADJP -> NP JJ, NP -> CD NNS, CD -> '61', NNS -> 'years', JJ -> 'old', , -> ',', S|<VP-.> -> VP ., VP -> MD VP, MD -> 'will', VP -> VB VP|<NP-PP-CLR-NP-TMP>, VB -> 'join', VP|<NP-PP-CLR-NP-TMP> -> NP VP|<PP-CLR-NP-TMP>, NP -> DT NN, DT -> 'the', NN -> 'board', VP|<PP-CLR-NP-TMP> -> PP-CLR NP-TMP, PP-CLR -> IN NP, IN -> 'as', NP -> DT NP|<JJ-NN>, DT -> 'a', NP|<JJ-NN> -> JJ NN, JJ -> 'nonexecutive', NN -> 'director', NP-TMP -> NNP CD, NNP -> 'nov.', CD -> '29', . -> '.'] <class 'nltk.grammar.Production'>


In [7]:
rules[0].rhs(), type(rules[0].rhs()[0])

((NP-SBJ, S|<VP-.>), nltk.grammar.Nonterminal)

In [8]:
rules[10].rhs(), type(rules[10].rhs()[0])

(('61',), str)

In [9]:
rules[0].lhs(), type(rules[0].lhs())

(S, nltk.grammar.Nonterminal)

In [10]:
print(cnf_train[0].leaves())

['pierre', 'vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', '29', '.']


ANSWER HERE
- productions():
- rhs():
- lhs():
- leaves():

## Question 2
To count the number of unique rules, nonterminals and terminals, pleaseimplement functions **collect_rules, collect_nonterminals, collect_terminals**


In [11]:
def collect_rules(train_data):
    '''
    Collect the rules that appear in data.
    params:
        train_data: list[Tree] --- list of Tree objects
    return:
        rules: list[nltk.grammar.Production] --- list of rules (Production objects)
        rules_counts: Counter object --- a dictionary that maps one rule (nltk.Nonterminal) to its number of 
                                         occurences (int) in train data.
    '''
    rules = list()
    rules_counts = Counter()
    ### YOUR CODE HERE (~ 2 lines)
    rules = [rule for tree in train_data for rule in tree.productions()]
    rule_counts = Counter(rules)
    ### YOUR CODE HERE
    return rules, rule_counts

def collect_nonterminals(rules):
    '''
    collect nonterminals that appear in the rules
    params:
        rules: list[nltk.grammar.Production] --- list of rules (Production objects)
    return:
        nonterminals: set(nltk.Nonterminal) --- set of nonterminals 
    '''
    nonterminals = list()
    ### YOUR CODE HERE (at least one line)
    nonterminals = [rule.lhs() for rule in rules if type(rule.lhs()) is Nonterminal]
    ### END OF YOUR CODE
    return set(nonterminals)

def collect_terminals(rules):
    '''
    collect terminals that appear in the rules
    params:
        rules: list[nltk.grammar.Production] --- list of rules (Production objects)
    return:
        terminals: set of strings --- set of terminals    
    '''
    terminals = list()
    ### YOUR CODE HERE (at least one line)
    terminals = [r for rule in rules for r in rule.rhs() if type(r) is str]
    ### END OF YOUR CODE

    return set(terminals)

In [12]:
train_rules, train_rules_counts = collect_rules(cnf_train)
nonterminals = collect_nonterminals(train_rules)
terminals = collect_terminals(train_rules)

In [13]:
### CORRECT ANSWER (196646, 31656, 11367, 7869)
len(train_rules), len(set(train_rules)), len(terminals), len(nonterminals)

(196646, 31656, 11367, 7869)

In [14]:
print(train_rules_counts.most_common(5))

[(, -> ',', 4876), (DT -> 'the', 4726), (. -> '.', 3814), (PP -> IN NP, 3273), (S|<VP-.> -> VP ., 3003)]


## Question 3
Implement the function **build_pcfg** which builds a dictionary that stores theterminal rules and nonterminal rules.

In [15]:
def build_pcfg(rules_counts):
    '''
    Build a dictionary that stores the terminal rules and nonterminal rules.
    param:
        rules_counts: Counter object --- a dictionary that maps one rule to its number of occurences in train data.
    return:
        rules_dict: dict(dict(dict)) --- a dictionary has a form like:
                    rules_dict = {'terminals':{'NP':{'the':1000,'an':500}, 'ADJ':{'nice':500,'good':100}},
                                  'nonterminals':{'S':{'NP@VP':1000},'NP':{'NP@NP':540}}}
    When building "rules_dict", you need to use "lhs()", "rhs()" funtion and convert Nonterminal to str.
    All the keys in the dictionary are of type str.
    '@' is used as a special symbol to split left and right nonterminal strings.
    '''
    
    rules_dict = dict()
    ### rules_dict['terminals'] contains rules like "NP->'the'"
    ### rules_dict['nonterminals'] contains rules like "S->NP@VP"
    rules_dict['terminals'] = defaultdict(dict)
    rules_dict['nonterminals'] = defaultdict(dict)
    
    ### YOUR CODE HERE
    for rule, counts in rules_counts.items():
        lhs = rule.lhs().symbol()
        rhs = rule.rhs()
        
        length = len(rhs)
        
        if length > 1:
            head, tail = rhs[0].symbol(), rhs[1].symbol()
            new_rhs = head + '@' + tail
            rules_dict['nonterminals'][lhs][new_rhs] = counts
        
        elif length == 1:
            if isinstance(rhs[0], str):
                new_rhs = rhs[0]
                rules_dict['terminals'][lhs][new_rhs] = counts
            else:
                raise ValueError
        else:
            raise ValueError
    ### END OF YOUR CODE
    return rules_dict

In [16]:
train_rules_dict = build_pcfg(train_rules_counts)

In [116]:
print(train_rules_dict['nonterminals']['S'])

{'NP-SBJ@S|<VP-.>': 1056, 'NP-SBJ-1@S|<VP-.>': 205, 'NP-SBJ+-NONE-@NP-PRD': 30, 'S-TPC-1@S|<,-NP-SBJ+NNS-VP-.>': 4, 'S-TPC-2@S|<,-NP-SBJ+NNS-VP-.>': 5, 'NP-SBJ+PRP@VP': 613, 'NP-SBJ+-NONE-@VP': 1354, 'SBAR-ADV@S|<,-NP-SBJ-VP-.>': 22, 'NP-SBJ-2@VP': 76, 'NP-SBJ+DT@VP': 22, 'NP-SBJ+PRP@S|<VP-.>': 130, 'NP-SBJ+NN@VP': 32, "NP-SBJ+EX@S|<VP-.-''>": 3, 'NP-SBJ+NNS@VP': 91, 'NP-SBJ-1+NN@VP': 3, 'S-TPC-1@S|<,-NP-SBJ-VP-.>': 45, 'PP-LOC@S|<,-NP-SBJ-VP-.>': 58, 'S-TPC-2@S|<,-NP-SBJ+PRP-VP-.>': 4, 'NP-SBJ-4+-NONE-@VP': 5, 'NP-SBJ@VP': 1073, 'NP-SBJ-5+-NONE-@VP': 4, 'S-TPC-2@S|<,-NP-SBJ-VP-.>': 20, 'PP-TMP@S|<,-NP-SBJ-VP-.>': 48, 'PP-TMP@S|<,-NP-SBJ-6-VP-.>': 1, 'NP-SBJ-7@S|<VP-.>': 1, 'NP-SBJ-8@VP': 1, 'NP-SBJ+NNS@S|<VP-.>': 51, '``@S|<CC-NP-SBJ-1+PRP-VP-.>': 1, 'NP-SBJ-9@S|<VP-.>': 1, 'NP-SBJ@S|<,-PP-,-VP-.>': 8, 'PP@S|<,-NP-SBJ-1+NNS-VP-.>': 2, 'ADVP+RB@S|<,-NP-SBJ-VP-.>': 26, 'S@S|<:-S-.>': 42, 'NP-SBJ-10@S|<VP-.>': 1, 'NP-SBJ+-NONE-@VP+VBD': 3, 'NP-SBJ-1+PRP@VP': 82, 'NP-SBJ+NNP@S|<VP-.>': 81

## Question 4
Estimate the probability of rule $NP\rightarrow NNP@NNP$

In [17]:
### ~0.0395
NUM = train_rules_dict['nonterminals']['NP']['NNP@NNP']
DEN = sum(train_rules_dict['nonterminals']['NP'].values())
NUM/DEN

0.03950843529348353

## Question 5
Find the terminal symbols in ''cnf_test[0]'' that never appeared in the PCFG we built. 

In [118]:
leaves = set(cnf_test[0].leaves())
leaves.difference(terminals)

{'constitutional-law'}

## Question 6
We can use smoothing techniques to handle these cases. A simple smoothing method is as follows. We first create a new "unknown" terminal symbol $unk$.

Next, for each original non-terminal symbol $A\in N$, we add one new rule $A \rightarrow unk$ to the original PCFG.

The smoothed probabilities for all rules can then be estimated as:
$$q_{smooth}(A \rightarrow \beta) = \frac {count(A \rightarrow \beta)}{count(A)+1}$$
$$q_{smooth}(A \rightarrow unk) = \frac {1}{count(A)+1}$$
where $|V|$ is the count of unique terminal symbols.


Implement the function **smooth_rules_prob** which returns the smoothed rule probabilities

In [21]:
def smooth_rules_prob(rules_counts):
    '''
    params:
        rules_counts: dict(dict(dict)) --- a dictionary has a form like:
                      rules_counts = {'terminals':{'NP':{'the':1000,'an':500}, 'ADJ':{'nice':500,'good':100}},
                                      'nonterminals':{'S':{'NP@VP':1000},'NP':{'NP@NP':540}}}
    
    return:
        rules_prob: dict(dict(dict)) --- a dictionary that has a form like:
                               rules_prob = {'terminals':{'NP':{'the':0.6,'an':0.3, '<unk>':0.1},
                                                          'ADJ':{'nice':0.6,'good':0.3,'<unk>':0.1},
                                                          'S':{'<unk>':0.01}}}
                                             'nonterminals':{'S':{'NP@VP':0.99}}
    '''
    rules_prob = copy.deepcopy(rules_counts)
    unk = '<unk>'
    ### Hint: don't forget to consider nonterminal symbols that appear only in rules_counts['nonterminals'].keys()
    ### YOUR CODE HERE
    for n in rules_counts['nonterminals'].keys():
        den = sum(rules_counts['nonterminals'][n].values())
        
        if n in rules_counts['terminals'].keys():
            den += sum(rules_counts['terminals'][n].values())
        
        for k,num in rules_counts['nonterminals'][n].items():
            rules_prob['nonterminals'][n][k] = float(num / (den+1))
        rules_prob['terminals'][n]['<unk>'] = 1.0 / (den+1)
        
    
    for t in rules_counts['terminals'].keys():
        den = sum(rules_counts['terminals'][t].values())
        
        if t in rules_counts['nonterminals'].keys():
            den += sum(rules_counts['nonterminals'][t].values())
        
        for k, num in rules_counts['terminals'][t].items():
            rules_prob['terminals'][t][k] = float(num / (den+1))
        rules_prob['terminals'][t]['<unk>'] = 1.0/(den+1)
    ### END OF YOUR CODE
    return rules_prob

In [22]:
s_rules_prob = smooth_rules_prob(train_rules_dict)
terminals.add('<unk>')

In [29]:
s_rules_prob['nonterminals']['NP']

{'NNP@NNP': 0.039506305917861376,
 'CD@NNS': 0.014013150803061335,
 'DT@NN': 0.10817074485286192,
 'DT@NP|<JJ-NN>': 0.03934461571628759,
 'NP@NP|<,-NP>': 0.0057130537889403905,
 'DT@NP|<NNP-VBG-NN>': 5.389673385792821e-05,
 'NP@PP': 0.08795946965613884,
 'JJ@NN': 0.020911932736876145,
 'NNP@NP|<NNP-NNP-NNP>': 0.003395494233049477,
 'DT@NP|<JJ-JJ-NN>': 0.0027487334267543386,
 'NNP@NP|<NN-NNS>': 0.00043117387086342567,
 'NP@NP|<PP-PP-LOC>': 0.0016707987495957746,
 'NN@NNS': 0.016384607092810175,
 'NP+NNS@RRC+VP': 5.389673385792821e-05,
 'QP@NNS': 0.004904602781071467,
 'DT@NP|<NN-NN>': 0.0168157809636736,
 'DT@NNS': 0.01924113398728037,
 'RB@NP|<JJ-NNS>': 0.00026948366928964106,
 'NP+NNS@SBAR': 0.0027487334267543386,
 'NP@NP|<PP-SBAR>': 0.0021019726204592,
 'ADJP@NP|<NNP-NNP>': 0.00010779346771585642,
 'NNP@NNS': 0.0021019726204592,
 'PRP$@NP|<NN-NN-NNS>': 0.00021558693543171284,
 'QP@NN': 0.003880564837770831,
 'NP@NP|<NNP-NNP-NNP>': 0.00043117387086342567,
 'NN@POS': 0.0010240379433006

In [32]:
print(s_rules_prob['nonterminals']['S']['NP-SBJ@S|<VP-.>'])
print(s_rules_prob['nonterminals']['S']['NP-SBJ-1@S|<VP-.>'])
print(s_rules_prob['nonterminals']['NP']['NNP@NNP'])
print(s_rules_prob['terminals']['NP'])

0.1300172371337109
0.025240088648116228
0.039506305917861376
{'<unk>': 5.389673385792821e-05}


In [122]:
len(terminals)

11368

## CKY Algorithm

Similar to the Viterbi algorithm, the CKY algorithm is a dynamic-programming algorithm. Given a PCFG $G=(N, \ \Sigma, \ S, \ R)$, we can use the CKY algorithm described in class to find the highest scoring parse tree for a sentence. 

First, let us complete the *CKY* function from scratch using only Python built-in functions and the Numpy package. 

The output should be two dictionaries $\pi$ and $bp$, which store the optimal probability and backpointer information respectively.

Given a sentence $w_0, w_1, ...,w_{n-1}$,  $\pi(i, k, X)$, $bp(i, k, X)$ refer to the highest score and backpointer for the (partial) parse tree that has the root X (a non-terminal symbol) and covers the word span $w_i, ..., w_{k-1}$, where $0 \le i < k \le n$. Note that a backpointer includes both the best grammar rule chosen and the best split point.
![tree](imgs/parse_tree.png)


## Question 7
Implement **CKY** function and run the test code to check your implementation.

In [41]:
def CKY(sent, rules_prob):
    '''
    params:
        sent: list[str] --- a list of strings
        rules_prob: dict(dict(dict)) --- a dictionary that has a form like:
                                           rules_prob = {'terminals':{'NP':{'the':0.6,'an':0.3, '<unk>':0.1},
                                                                      'ADJ':{'nice':0.6,'good':0.3,'<unk>':0.1},
                                                                      'S':{'<unk>':0.01}}}
                                                         'nonterminals':{'S':{'NP@VP':0.99}}
    return:
        score: dict() --- score[(i,i+span)][X] represents the highest score for the (partial) parse tree that has the root X
                          across words w_i, w_{i+1},..., w_{i+span-1}.
        back: dict() --- back[(i,i+span)][root] = (split , left_child, right_child); split: int; 
                         left_child: str; right_child: str. 
    '''
    score = defaultdict(dict)
    back = defaultdict(dict)
    sent_len = len(sent)
    
    for i in range(sent_len):
        for x in rules_prob['terminals'].keys():
            score[(i, i+1)][x] = rules_prob['terminals'][x].get(sent[i], 0)
            if score[(i, i+1)][x] > 0:
                back[(i, i+1)][x] = (None, sent[i],  None)

#     for span in range(1, sent_len):
#         for begin in range(1, sent_len - span + 1):
#             end = begin + span
#             for split in range(begin, end):
#                 for x in rules_prob['nonterminals'].keys():
#                     for y, p in rules_prob['nonterminals'][x].items():
#                         [lhs, rhs] = y.split('@')
#                         prob = p * score[(begin-1, split)].get(lhs, 0) * score[(split, end)].get(rhs, 0)
#                         if prob > score[(begin-1, end)].get(x, 0):
#                             score[(begin-1, end)][x] = prob
#                             back[(begin-1, end)][x] = (split, lhs, rhs)
                            
    for span in range(2, sent_len+1):
        for begin in range(0, sent_len - span + 1):
            end = begin + span 
            for split in range(begin+1, end):
                for x in rules_prob['nonterminals'].keys():
                    for y, p in rules_prob['nonterminals'][x].items():
                        [lhs, rhs] = y.split('@')
                        prob = p * score[(begin, split)].get(lhs, 0) * score[(split, end)].get(rhs, 0)
                        if prob > score[(begin, end)].get(x, 0):
                            score[(begin, end)][x] = prob
                            back[(begin, end)][x] = (split, lhs, rhs)
                            
    return score, back

In [42]:
sent = cnf_train[0].leaves()
score, back = CKY(sent, s_rules_prob)

In [43]:
# The value should be aroun 
score[(0, len(sent))]['S']

9.135335125206641e-52

In [25]:
# The value should be aroun 
score[(0, len(sent))]['S']

9.135335125206641e-52

## Question 8
Implement **build_tree** function according to algorithm 2 to reconstruct theparse tree

In [36]:
def build_tree(back, root):
    begin = root[0]
    end = root[1]
    root_label = root[2]
    
    split, left_child, right_child = back[(begin,end)][root_label]

    if right_child:
        left_root = (begin, split, left_child)
        right_root = (split, end, right_child)
        
        left_tree = build_tree(back, left_root)
        right_tree = build_tree(back, right_root)
        
        tree = nltk.tree.Tree(root_label, [left_tree, right_tree])
    else :
        tree = nltk.tree.Tree(root_label, [left_child])
        
    return tree

In [37]:
build_tree(back, (0, len(sent), 'S')).pprint()

(S
  (NP-SBJ
    (NP (NNP pierre) (NNP vinken))
    (NP-SBJ|<,-NP-,>
      (, ,)
      (NP-SBJ|<NP-,>
        (NP (CD 61) (NP|<NNS-JJ> (NNS years) (JJ old)))
        (, ,))))
  (S|<VP-.>
    (VP
      (MD will)
      (VP
        (VB join)
        (VP|<NP-PP-CLR-NP-TMP>
          (NP (DT the) (NN board))
          (VP|<PP-CLR-NP-TMP>
            (PP-CLR
              (IN as)
              (NP
                (DT a)
                (NP|<JJ-NN> (JJ nonexecutive) (NN director))))
            (NP-TMP (NNP nov.) (CD 29))))))
    (. .)))


## Question 9

In [44]:
def set_leave_index(tree):
    '''
    Label the leaves of the tree with indexes
    Arg:
        tree: original tree, nltk.tree.Tree
    Return:
        tree: preprocessed tree, nltk.tree.Tree
    '''
    for idx, _ in enumerate(tree.leaves()):
        tree_location = tree.leaf_treeposition(idx)
        non_terminal = tree[tree_location[:-1]]
        non_terminal[0] = non_terminal[0] + "_" + str(idx)
    return tree

def get_nonterminal_bracket(tree):
    '''
    Obtain the constituent brackets of a tree
    Arg:
        tree: tree, nltk.tree.Tree
    Return:
        nonterminal_brackets: constituent brackets, set
    '''
    nonterminal_brackets = set()
    for tr in tree.subtrees():
        label = tr.label()
        #print(tr.leaves())
        if len(tr.leaves()) == 0:
            continue
        start = tr.leaves()[0].split('_')[-1]
        end = tr.leaves()[-1].split('_')[-1]
        if start != end:
            nonterminal_brackets.add(label+'-('+start+':'+end+')')
    return nonterminal_brackets

def word2lower(w, terminals):
    '''
    Map an unknow word to "unk"
    '''
    return w.lower() if w in terminals else '<unk>'

In [129]:
correct_count = 0
pred_count = 0
gold_count = 0
for i, t in enumerate(cnf_test):
    #Protect the original tree 
    t = copy.deepcopy(t)
    sent = t.leaves()  
    #Map the unknow words to "unk"
    sent = [word2lower(w.lower(), terminals) for w in sent]
    
    #CKY algorithm
    score, back = CKY(sent, s_rules_prob)
    candidate_tree = build_tree(back, (0, len(sent), 'S'))
    
    #Extract constituents from the gold tree and predicted tree
    pred_tree = set_leave_index(candidate_tree)
    pred_brackets = get_nonterminal_bracket(pred_tree)
    
    #Count correct constituents
    pred_count += len(pred_brackets)
    gold_tree = set_leave_index(t)
    gold_brackets = get_nonterminal_bracket(gold_tree)
    gold_count += len(gold_brackets)
    current_correct_num = len(pred_brackets.intersection(gold_brackets))
    correct_count += current_correct_num
    
    print('#'*20)
    print('Test Tree:', i+1)
    print('Constituent number in the predicted tree:', len(pred_brackets))
    print('Constituent number in the gold tree:', len(gold_brackets))
    print('Correct constituent number:', current_correct_num)

recall = correct_count/gold_count
precision = correct_count/pred_count
f1 = 2*recall*precision/(recall+precision)

####################
Test Tree: 1
Constituent number in the predicted tree: 20
Constituent number in the gold tree: 20
Correct constituent number: 14
####################
Test Tree: 2
Constituent number in the predicted tree: 54
Constituent number in the gold tree: 54
Correct constituent number: 30
####################
Test Tree: 3
Constituent number in the predicted tree: 30
Constituent number in the gold tree: 30
Correct constituent number: 23
####################
Test Tree: 4
Constituent number in the predicted tree: 17
Constituent number in the gold tree: 17
Correct constituent number: 16
####################
Test Tree: 5
Constituent number in the predicted tree: 32
Constituent number in the gold tree: 32
Correct constituent number: 26
####################
Test Tree: 6
Constituent number in the predicted tree: 40
Constituent number in the gold tree: 40
Correct constituent number: 18
####################
Test Tree: 7
Constituent number in the predicted tree: 22
Constituent number in

In [46]:
print('Overall precision: {:.3f}, recall: {:.3f}, f1: {:.3f}'.format(precision, recall, f1))

Overall precision: 0.545, recall: 0.545, f1: 0.545
